In [1]:
import numpy as np

In [2]:
data_train = np.loadtxt('hw6_train.dat')
data_test = np.loadtxt('hw6_test.dat')

In [3]:
print(data_train)
print(data_train.shape, '\n')
print(data_test)
print(data_test.shape)

[[ 3.95632268  1.80130232 -0.560404   ...  1.15647158 -0.02457943
   1.        ]
 [ 1.32397219 -2.40197608  2.00437315 ... -0.66552085 -0.2635734
   1.        ]
 [ 1.41287312  2.51772309  1.81555809 ... -0.13231329 -0.41357255
  -1.        ]
 ...
 [-3.54241785  1.90927013  0.03063317 ... -0.69957505  0.19560916
   1.        ]
 [ 1.46852162 -2.35922982 -0.2427209  ... -0.57478285  0.23606362
   1.        ]
 [ 3.45467048 -1.04328297  1.87407764 ... -0.51747156  1.1260002
   1.        ]]
(1000, 11) 

[[-1.07980097 -1.28844866 -0.96921961 ... -0.07886518 -0.87643651
   1.        ]
 [ 2.40438778 -3.32292852  0.66547387 ...  0.47456144 -0.62825596
  -1.        ]
 [-0.54582793  0.77878532  0.08592099 ...  1.21068115 -0.25649693
  -1.        ]
 ...
 [ 2.24361791  1.92961334 -0.32604635 ...  0.10112766 -0.87566657
   1.        ]
 [ 3.33904654  1.92063043 -1.55614559 ... -0.51534274  1.11347745
  -1.        ]
 [-1.71472981 -0.67902764  0.37347159 ...  0.38208181 -0.11064126
   1.        ]]
(1000

In [4]:
class Question:
    def __init__(self, feature, theta):
        self.feature = feature
        self.theta = theta

    def match(self, example):
        return example[self.feature] > self.theta
    
    def __repr__(self):
        return "Is feature %s > %s ?" % (self.feature, str(self.theta))

In [5]:
def partition(rows, question):  
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [6]:
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [7]:
def classification_error(rows):

    counts = class_counts(rows)
    impurity = 1
    majority = max(counts, key=counts.get)
    prob = counts[majority] / float(len(rows))
    impurity -= prob
    return impurity

In [8]:
def gini(rows):

    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [9]:
def get_thetas(rows, col):
    values =  sorted(set([row[col] for row in rows]))
    thetas = []
    for i in range(len(values)-1):
        thetas.append( (values[i]+values[i+1])/2 )
    return thetas

In [11]:
def find_best_split(rows, impurity_function):
    best_error = len(rows)
    best_question = None
    n_features = len(rows[0]) - 1

    for col in range(n_features):
        thetas = get_thetas(rows, col)
        for val in thetas:
            question = Question(col, val)
            true_rows, false_rows = partition(rows, question)

            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            error = len(true_rows)*impurity_function(true_rows) + len(false_rows)*impurity_function(false_rows)
            if(error < best_error):
                best_error, best_question = error, question

    return best_error, best_question

In [12]:
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [13]:
class Decision_Node:
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [14]:
def build_tree(rows, impurity_function):
    # Base case, one class
    if (len(class_counts(rows)) == 1):
        return Leaf(rows)

    err, question = find_best_split(rows, impurity_function)

    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows, impurity_function)
    false_branch = build_tree(false_rows, impurity_function)

    return Decision_Node(question, true_branch, false_branch)

In [94]:
class Decision_Tree:
    def __init__(self, root=None):
        self.root = root
    def check_root(self):
        return self.root==None
    def fit(self, rows, impurity_function):
        if (len(class_counts(rows)) == 1):
            self.root = Leaf(rows)
        else:
            self.root = build_tree(rows, impurity_function)
    
    def _classify(self, row, node):
        if isinstance(node, Leaf):
            return node.predictions
        if node.question.match(row):
            return self._classify(row, node.true_branch)
        else:
            return self._classify(row, node.false_branch)
    
    def predict(self, rows):
        pr = []
        for row in rows:
            cl = self._classify(row, self.root)
            pr.append(max(cl, key=cl.get))
        return pr
    
    def score(self, rows):
        wrong = 0
        for row in rows:
            cl = self._classify(row, self.root)
            pr = max(cl, key=cl.get)
            if (row[-1] != pr):
                wrong += 1
        return wrong/len(rows)
    
    def _print_tree(self, node, spacing=""):
        if isinstance(node, Leaf):
            print (spacing + "Predict", node.predictions)
            return
        
        print (spacing + str(node.question))

        print (spacing + '--> True:')
        self._print_tree(node.true_branch, spacing + "  ")

        print (spacing + '--> False:')
        self._print_tree(node.false_branch, spacing + "  ")
        
    def print_tree(self):
        self._print_tree(self.root)

In [121]:
dt = Decision_Tree()
dt.fit(data_train, gini)
dt.score(data_test)

0.166

In [129]:
dt.predict(data_test)

[1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,

In [132]:
dt.print_tree()

Is feature 5 > -1.3180367635499999 ?
--> True:
  Is feature 9 > 0.97445967225 ?
  --> True:
    Is feature 2 > 1.82562346345 ?
    --> True:
      Is feature 0 > 0.23738037289999997 ?
      --> True:
        Is feature 5 > 0.48950835115 ?
        --> True:
          Predict {-1.0: 3}
        --> False:
          Predict {1.0: 9}
      --> False:
        Predict {-1.0: 7}
    --> False:
      Is feature 4 > -1.33348060355 ?
      --> True:
        Is feature 0 > -1.4231283196 ?
        --> True:
          Predict {-1.0: 55}
        --> False:
          Is feature 0 > -1.6852853846 ?
          --> True:
            Predict {1.0: 1}
          --> False:
            Is feature 2 > -1.4076548431 ?
            --> True:
              Predict {-1.0: 22}
            --> False:
              Is feature 0 > -2.8119415982 ?
              --> True:
                Predict {1.0: 1}
              --> False:
                Predict {-1.0: 3}
      --> False:
        Is feature 7 > 0.0983719705 ?
    

In [130]:
np.savetxt("pred.csv",  
           dt.predict(data_test), 
           delimiter =", ",  
           fmt ='% s')

In [66]:
from random import choices

In [103]:
class Random_forest:
    def __init__(self, n_trees=2000):
        self.n_trees = n_trees
        self.trees = []
        self.train = None
        self.oob_i_tree = [] # for Q18
    
    def fit(self, rows):
        self.train = np.array(rows)
        self.oob_i_tree = [[] for i in range(len(self.train))]
        for i in range(self.n_trees):
            # bootstrapping
            index = [i for i in range(len(self.train))]
            i_bag = choices(index, k=int(len(self.train)*0.5))
            i_oob = [item for item in index if item not in i_bag]
            
            # for Q18
            for j in i_oob:
                self.oob_i_tree[j].append(i)
            bag = self.train[i_bag]
            
            dt = Decision_Tree()
            dt.fit(bag, gini)

            self.trees.append(dt)

    # for Q15
    def avg_score(self, rows):
        score = 0
        for tree in self.trees:
            score += tree.score(rows)
        return (score/self.n_trees)
    
    # for Q16, 17
    def score(self, rows):
        sc = np.array([0.0 for i in range(len(rows))])
        for tree in self.trees:
            sc += np.array(tree.predict(rows))
        for n, i in enumerate(sc):
            if i > 0:
                sc[n] = 1.0
            else:
                sc[n] = -1.0
        
        wrong = 0
        for tr, pr in zip(rows, sc):
            if (tr[-1] != pr):
                wrong += 1
        
        return wrong/len(rows)
    
    def oob_score(self):
        error = 0
        for i in range(len(self.train)):
            if self.oob_i_tree[i] == []:
                error += int(self.train[i][-1] == -1)
                continue
            
            pr = 0
            for j in self.oob_i_tree[i]:
                pr += self.trees[j].predict([self.train[i]])[0]
            if pr > 0:
                pr = 1
            else:
                pr = -1
            error += int(self.train[i][-1] == pr)
        return error/len(self.train)

In [133]:
rf = Random_forest(2000)
rf.fit(data_train)
print(rf.avg_score(data_test))
print(rf.score(data_train))
print(rf.score(data_test))
print(rf.oob_score())

0.2616239999999999
0.009
0.153
0.92
